In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
%matplotlib inline

In [33]:
def osamples(n_samples):
    nfpa = np.random.normal(loc=26.42496428571428, scale=0.7598622819300876, size=n_samples)
    nfpa=np.round(nfpa).astype(int)
    gafpa=nfpa*16/28
    gafpa=np.round(gafpa).astype(int)
    ofpa=nfpa-gafpa
    return ofpa, gafpa
osamples(2)
#38.757275 1.5456876137742062

(array([11, 12]), array([15, 15]))